In [255]:
#!/usr/bin/env python3

import pandas as pd
import errno
import os, sys
import numpy as np

'''
Author: Enrico Ceccolini
    Given the raw data of Eurora (GPU_MEASUREMENT and MIC_MEASUREMENT)
    and a time interval, it returns only the measurements
    of that period, sampled at 5 sec intervals
'''

datadir = "/datasets/eurora_data/db_local/"
#datadir = "/datasets/eurora_data/db/"

### select an interval from
## 1 settings wholeData
interval_comment_whole = "WholeData"
start_time_whole = pd.to_datetime('2014-03-24')
end_time_whole = pd.to_datetime('2015-08-12')

## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')

suffix = "_5sec_"
# settings

global_start_time = pd.to_datetime('2014-03-24')
global_end_time = pd.to_datetime('2015-08-12')

#gpu_nodes = ['33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
gpu_nodes = ['58', '59', '60', '61', '62', '63', '64']
mic_nodes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
mic_nodes = ['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
#mic_nodes = ['02', '10']
#node = "33"
#infile = datadir + "gpu/gpu" + node + ".csv"

## read dataset
#whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
#print("there are {} instances in the whole dataset".format(whole_node_power_data.shape[0]))

In [36]:
print(node)
infile = datadir + "gpu/gpu" + node + ".csv"

# read dataset
whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
print("data rows {}".format(whole_node_power_data.shape))

# drop useless columns
whole_node_power_data = whole_node_power_data.drop([str(node)], axis=1)

# drop the duplicates rows (same timestamp, cpu_id, ...)
whole_node_power_data = whole_node_power_data.drop_duplicates() 
print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))

33
data rows (6090117, 4)
data rows after drop_duplicates 5302089


In [37]:
whole_node_power_data.head(5)

,gpu_id,timestamp,pow
0,0,2014-03-24 13:56:30,13.191
1,1,2014-03-24 13:56:30,13.211
2,0,2014-03-24 13:56:35,13.515
3,1,2014-03-24 13:56:35,13.443
4,0,2014-03-24 13:56:40,13.515


In [38]:
# select data in the specified interval
interval_data = whole_node_power_data

In [39]:
# split the data referring to the gpu0 and gpu1
gpu0 = interval_data[interval_data['gpu_id'] == 0]
gpu1 = interval_data[interval_data['gpu_id'] == 1]
print("data row after split: {}".format(gpu0.shape[0]))
print("data row after split: {}".format(gpu1.shape[0]))

data row after split: 2651038
data row after split: 2651051


In [40]:
# drop the duplicates rows 
gpu0 = gpu0.drop_duplicates(subset=['timestamp'])
gpu1 = gpu1.drop_duplicates(subset=['timestamp'])
print("data row after split: {}".format(gpu0.shape[0]))
print("data row after split: {}".format(gpu1.shape[0]))

data row after split: 2650945
data row after split: 2650961


In [41]:
if gpu0.shape[0] < gpu1.shape[0]:
    gpu1 = gpu1[gpu1.timestamp >= gpu0['timestamp'].iloc[0]]
if gpu1.shape[0] < gpu0.shape[0]:
    gpu0 = gpu0[gpu0.timestamp >= gpu1['timestamp'].iloc[0]]
print("data row after split: {}".format(gpu0.shape[0]))
print("data row after split: {}".format(gpu1.shape[0]))

data row after split: 2650945
data row after split: 2650961


In [42]:
gpu0.timestamp = pd.to_datetime(gpu0.timestamp)
gpu1.timestamp = pd.to_datetime(gpu1.timestamp)
gpu0 = gpu0.set_index(["timestamp"])
gpu1 = gpu1.set_index(["timestamp"])
gpu0.head(5)

,gpu_id,pow
timestamp,,
2014-03-24 13:56:30,0,13.191
2014-03-24 13:56:35,0,13.515
2014-03-24 13:56:40,0,13.515
2014-03-24 13:56:45,0,13.495
2014-03-24 13:56:50,0,13.535


In [43]:
# resample data to 5 sec
gpu0_5sec = gpu0.resample('5S').mean()
gpu1_5sec = gpu1.resample('5S').mean()

In [44]:
gpu0_5sec = gpu0_5sec.drop(['gpu_id'], axis=1)
gpu1_5sec = gpu1_5sec.drop(['gpu_id'], axis=1)

In [45]:
gpu0_5sec.head(1)

,pow
timestamp,
2014-03-24 13:56:30,13.191


In [46]:
first_date = gpu0_5sec.index[0]
last_date = gpu0_5sec.index[gpu0_5sec.shape[0]-1]
index_before = pd.date_range(start=global_start_time, end=first_date - np.timedelta64(5, 's'), freq='5S')
index_before

DatetimeIndex(['2014-03-24 00:00:00', '2014-03-24 00:00:05',
               '2014-03-24 00:00:10', '2014-03-24 00:00:15',
               '2014-03-24 00:00:20', '2014-03-24 00:00:25',
               '2014-03-24 00:00:30', '2014-03-24 00:00:35',
               '2014-03-24 00:00:40', '2014-03-24 00:00:45',
               ...
               '2014-03-24 13:55:40', '2014-03-24 13:55:45',
               '2014-03-24 13:55:50', '2014-03-24 13:55:55',
               '2014-03-24 13:56:00', '2014-03-24 13:56:05',
               '2014-03-24 13:56:10', '2014-03-24 13:56:15',
               '2014-03-24 13:56:20', '2014-03-24 13:56:25'],
              dtype='datetime64[ns]', length=10038, freq='5S')

In [47]:
index_after = pd.date_range(start=last_date + np.timedelta64(5, 's'), end=global_end_time, freq='5s')
index_after

DatetimeIndex(['2015-04-18 18:43:00', '2015-04-18 18:43:05',
               '2015-04-18 18:43:10', '2015-04-18 18:43:15',
               '2015-04-18 18:43:20', '2015-04-18 18:43:25',
               '2015-04-18 18:43:30', '2015-04-18 18:43:35',
               '2015-04-18 18:43:40', '2015-04-18 18:43:45',
               ...
               '2015-08-11 23:59:15', '2015-08-11 23:59:20',
               '2015-08-11 23:59:25', '2015-08-11 23:59:30',
               '2015-08-11 23:59:35', '2015-08-11 23:59:40',
               '2015-08-11 23:59:45', '2015-08-11 23:59:50',
               '2015-08-11 23:59:55', '2015-08-12 00:00:00'],
              dtype='datetime64[ns]', length=1991005, freq='5S')

In [48]:
padding_before = pd.DataFrame(index=index_before, columns=['pow'])
padding_before.index.name = 'timestamp'
#padding_before
padding_after = pd.DataFrame(index=index_after, columns=['pow'])
padding_after.index.name = 'timestamp'
# padding_after

In [49]:
gpu0_5sec = pd.concat([padding_before, gpu0_5sec])
gpu1_5sec = pd.concat([padding_before, gpu1_5sec])
gpu0_5sec = pd.concat([gpu0_5sec, padding_after])
gpu1_5sec = pd.concat([gpu1_5sec, padding_after])

In [50]:
gpu0_5sec

,pow
timestamp,
2014-03-24 00:00:00,NaN
2014-03-24 00:00:05,NaN
2014-03-24 00:00:10,NaN
2014-03-24 00:00:15,NaN
2014-03-24 00:00:20,NaN
2014-03-24 00:00:25,NaN
2014-03-24 00:00:30,NaN
2014-03-24 00:00:35,NaN
2014-03-24 00:00:40,NaN


In [51]:
gpu0_5sec = gpu0_5sec.rename(index=str, columns={"pow": "pow_0"})
gpu1_5sec = gpu1_5sec.rename(index=str, columns={"pow": "pow_1"})
# cpu0_1min.head(1)

In [52]:
# concat
node_5sec = pd.concat([gpu0_5sec, gpu1_5sec], axis=1)
node_5sec.head(1)

,pow_0,pow_1
timestamp,,
2014-03-24 00:00:00,NaN,NaN


In [53]:
node_5sec['pow_tot'] = node_5sec['pow_0'] + node_5sec['pow_1']
node_5sec = node_5sec.drop(['pow_0', 'pow_1'], axis=1)
node_5sec.head(1)

,pow_tot
timestamp,
2014-03-24 00:00:00,NaN


In [55]:
interval_data_jobs = node_5sec[pd.to_datetime(node_5sec.index) >= start_time]
interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs.index) <= end_time]
print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))
interval_data_jobs.head(10)

interval_data_jobs contains 3715201 records


,pow_tot
timestamp,
2014-03-31 00:00:00,44.248
2014-03-31 00:00:05,44.261
2014-03-31 00:00:10,44.275
2014-03-31 00:00:15,44.328
2014-03-31 00:00:20,44.341
2014-03-31 00:00:25,44.341
2014-03-31 00:00:30,44.328
2014-03-31 00:00:35,44.328
2014-03-31 00:00:40,44.328


In [58]:
# write
interval_comment = "Alina"
outfile = datadir + 'gpu/gpu' + node + "_5sec_" + interval_comment + ".csv"
interval_data_jobs.to_csv(outfile)

In [68]:
for node in gpu_nodes:
    print(node)
    infile = datadir + "gpu/gpu" + node + ".csv"

    # read dataset
    whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
    print("data rows {}".format(whole_node_power_data.shape))

    # drop useless columns
    whole_node_power_data = whole_node_power_data.drop([str(node)], axis=1)

    # drop the duplicates rows (same timestamp, cpu_id, ...)
    whole_node_power_data = whole_node_power_data.drop_duplicates() 
    print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))
    
    
    interval_data = whole_node_power_data
    
    
    # split the data referring to the gpu0 and gpu1
    gpu0 = interval_data[interval_data['gpu_id'] == 0]
    gpu1 = interval_data[interval_data['gpu_id'] == 1]
    print("data row after split: {}".format(gpu0.shape[0]))
    print("data row after split: {}".format(gpu1.shape[0]))
    
    
    # drop the duplicates rows 
    gpu0 = gpu0.drop_duplicates(subset=['timestamp'])
    gpu1 = gpu1.drop_duplicates(subset=['timestamp'])
    print("data row after split: {}".format(gpu0.shape[0]))
    print("data row after split: {}".format(gpu1.shape[0]))
    
    
    if gpu0.shape[0] < gpu1.shape[0]:
        gpu1 = gpu1[gpu1.timestamp >= gpu0['timestamp'].iloc[0]]
    if gpu1.shape[0] < gpu0.shape[0]:
        gpu0 = gpu0[gpu0.timestamp >= gpu1['timestamp'].iloc[0]]
    print("data row after split: {}".format(gpu0.shape[0]))
    print("data row after split: {}".format(gpu1.shape[0]))
    
    
    gpu0.timestamp = pd.to_datetime(gpu0.timestamp)
    gpu1.timestamp = pd.to_datetime(gpu1.timestamp)
    gpu0 = gpu0.set_index(["timestamp"])
    gpu1 = gpu1.set_index(["timestamp"])
    
    
    # resample data to 5 sec
    gpu0_5sec = gpu0.resample('5S').mean()
    gpu1_5sec = gpu1.resample('5S').mean()
    
    
    gpu0_5sec = gpu0_5sec.drop(['gpu_id'], axis=1)
    gpu1_5sec = gpu1_5sec.drop(['gpu_id'], axis=1)
    
    
    first_date = gpu0_5sec.index[0]
    last_date = gpu0_5sec.index[gpu0_5sec.shape[0]-1]
    index_before = pd.date_range(start=global_start_time, end=first_date - np.timedelta64(5, 's'), freq='5S')
    
    
    index_after = pd.date_range(start=last_date + np.timedelta64(5, 's'), end=global_end_time, freq='5s')
    
    
    padding_before = pd.DataFrame(index=index_before, columns=['pow'])
    padding_before.index.name = 'timestamp'
    #padding_before
    padding_after = pd.DataFrame(index=index_after, columns=['pow'])
    padding_after.index.name = 'timestamp'
    # padding_after
    
    
    gpu0_5sec = pd.concat([padding_before, gpu0_5sec])
    gpu1_5sec = pd.concat([padding_before, gpu1_5sec])
    gpu0_5sec = pd.concat([gpu0_5sec, padding_after])
    gpu1_5sec = pd.concat([gpu1_5sec, padding_after])
    
    
    gpu0_5sec = gpu0_5sec.rename(index=str, columns={"pow": "pow_0"})
    gpu1_5sec = gpu1_5sec.rename(index=str, columns={"pow": "pow_1"})
    
    
    # concat
    node_5sec = pd.concat([gpu0_5sec, gpu1_5sec], axis=1)
    
    
    node_5sec['pow_tot'] = node_5sec['pow_0'] + node_5sec['pow_1']
    node_5sec = node_5sec.drop(['pow_0', 'pow_1'], axis=1)
    
    
    interval_data_jobs = node_5sec[pd.to_datetime(node_5sec.index) >= start_time]
    interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs.index) <= end_time]
    print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))
    
    
    # write
    interval_comment = "Alina"
    outfile = datadir + 'gpu/gpu' + node + "_5sec_" + interval_comment + ".csv"
    interval_data_jobs.to_csv(outfile)

58
data rows (7995050, 4)
data rows after drop_duplicates 6700359
data row after split: 3350202
data row after split: 3350157
data row after split: 3350047
data row after split: 3350003
data row after split: 3350047
data row after split: 3350003
interval_data_jobs contains 3715201 records
59
data rows (7361869, 4)
data rows after drop_duplicates 6043383
data row after split: 3021693
data row after split: 3021690
data row after split: 3021566
data row after split: 3021563
data row after split: 3021566
data row after split: 3021563
interval_data_jobs contains 3715201 records
60
data rows (7198613, 4)
data rows after drop_duplicates 6057383
data row after split: 3028712
data row after split: 3028671
data row after split: 3028536
data row after split: 3028497
data row after split: 3028536
data row after split: 3028497
interval_data_jobs contains 3715201 records
61
data rows (7787681, 4)
data rows after drop_duplicates 6469428
data row after split: 3234749
data row after split: 3234679
data

In [304]:
for node in mic_nodes:
    print(node)
    infile = datadir + "mic/mic" + node + ".csv"

    # read dataset
    whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
    print("data rows {}".format(whole_node_power_data.shape))

    # drop useless columns
    whole_node_power_data = whole_node_power_data.drop([str(node)], axis=1)

    # drop the duplicates rows (same timestamp, cpu_id, ...)
    whole_node_power_data = whole_node_power_data.drop_duplicates() 
    print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))
    
    
    interval_data = whole_node_power_data
    
    
    # split the data referring to the mic0 and mic1
    mic0 = interval_data[interval_data['mic_id'] == 0]
    mic1 = interval_data[interval_data['mic_id'] == 1]
    print("data row after split: {}".format(mic0.shape[0]))
    print("data row after split: {}".format(mic1.shape[0]))
    
    
    # drop the duplicates rows 
    mic0 = mic0.drop_duplicates(subset=['timestamp'])
    mic1 = mic1.drop_duplicates(subset=['timestamp'])
    print("data row after split: {}".format(mic0.shape[0]))
    print("data row after split: {}".format(mic1.shape[0]))
    
    
    if mic0['timestamp'].iloc[0] < mic1['timestamp'].iloc[0]:
        mic0 = mic0[mic0.timestamp >= mic1['timestamp'].iloc[0]]
    if mic1['timestamp'].iloc[0] < mic0['timestamp'].iloc[0]:
        mic1 = mic1[mic1.timestamp >= mic0['timestamp'].iloc[0]]
        
    #if mic0.shape[0] < mic1.shape[0]:
    #    mic1 = mic1[mic1.timestamp >= mic0['timestamp'].iloc[0]]
    #if mic1.shape[0] < mic0.shape[0]:
    #    mic0 = mic0[mic0.timestamp >= mic1['timestamp'].iloc[0]]
    print("data row after split: {}".format(mic0.shape[0]))
    print("data row after split: {}".format(mic1.shape[0]))
    
    if mic0['timestamp'].iloc[-1] < mic1['timestamp'].iloc[-1]:
        mic1 = mic1[mic1.timestamp <= mic0['timestamp'].iloc[-1]]
    if mic1['timestamp'].iloc[-1] < mic0['timestamp'].iloc[-1]:
        mic0 = mic0[mic0.timestamp <= mic1['timestamp'].iloc[-1]]
    print("data row after split: {}".format(mic0.shape[0]))
    print("data row after split: {}".format(mic1.shape[0]))

    
    mic0.timestamp = pd.to_datetime(mic0.timestamp)
    mic1.timestamp = pd.to_datetime(mic1.timestamp)
    mic0 = mic0.set_index(["timestamp"])
    mic1 = mic1.set_index(["timestamp"])
    
    
    # resample data to 5 sec
    mic0_5sec = mic0.resample('5S').mean()
    mic1_5sec = mic1.resample('5S').mean()
    
    
    mic0_5sec = mic0_5sec.drop(['mic_id'], axis=1)
    mic1_5sec = mic1_5sec.drop(['mic_id'], axis=1)
    

    
    first_date = mic0_5sec.index[0]
    last_date = mic0_5sec.index[mic0_5sec.shape[0]-1]
    index_before = pd.date_range(start=global_start_time, end=first_date - np.timedelta64(5, 's'), freq='5S')
    index_after = pd.date_range(start=last_date + np.timedelta64(5, 's'), end=global_end_time, freq='5s')
    padding_before = pd.DataFrame(index=index_before, columns=['total_power'])
    padding_before.index.name = 'timestamp'
    #padding_before
    padding_after = pd.DataFrame(index=index_after, columns=['total_power'])
    padding_after.index.name = 'timestamp'
    # padding_after

    mic0_5sec = pd.concat([padding_before, mic0_5sec])
    mic1_5sec = pd.concat([padding_before, mic1_5sec])
    mic0_5sec = pd.concat([mic0_5sec, padding_after])
    mic1_5sec = pd.concat([mic1_5sec, padding_after])
    
    mic0_5sec = mic0_5sec[~mic0_5sec.index.duplicated()]
    mic1_5sec = mic1_5sec[~mic1_5sec.index.duplicated()]
    print("data row after padding: {}".format(mic0_5sec.shape[0]))
    print("data row after padding: {}".format(mic1_5sec.shape[0]))
    
    
    #mic0_5sec = mic0_5sec.loc[mic0_5sec.index.drop_duplicates()]
    #mic1_5sec = mic1_5sec.loc[mic1_5sec.index.drop_duplicates()]
    #print("data row after padding: {}".format(mic0_5sec.shape[0]))
    #print("data row after padding: {}".format(mic1_5sec.shape[0]))
    
    
    mic0_5sec = mic0_5sec.rename(index=str, columns={"total_power": "pow_0"})
    mic1_5sec = mic1_5sec.rename(index=str, columns={"total_power": "pow_1"})
    

                       
    # concat
    node_5sec = pd.concat([mic0_5sec, mic1_5sec], axis=1)
    
    
    node_5sec['pow_tot'] = node_5sec['pow_0'] + node_5sec['pow_1']
    node_5sec = node_5sec.drop(['pow_0', 'pow_1'], axis=1)
    
    
    interval_data_jobs = node_5sec[pd.to_datetime(node_5sec.index) >= start_time]
    interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs.index) <= end_time]
    print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))
    
    
    # write
    interval_comment = "Alina"
    outfile = datadir + 'mic/mic' + node + "_5sec_" + interval_comment + ".csv"
    interval_data_jobs.to_csv(outfile)

13
data rows (647583, 4)
data rows after drop_duplicates 630141
data row after split: 298615
data row after split: 331526
data row after split: 298612
data row after split: 331522
data row after split: 298612
data row after split: 331522
data row after split: 297633
data row after split: 331522
data row after padding: 8743681
data row after padding: 8743681
interval_data_jobs contains 3715201 records
14
data rows (1319594, 4)
data rows after drop_duplicates 581220
data row after split: 317333
data row after split: 263887
data row after split: 116937
data row after split: 105646
data row after split: 116937
data row after split: 105646
data row after split: 116937
data row after split: 105646
data row after padding: 8743681
data row after padding: 8743681
interval_data_jobs contains 3715201 records
15
data rows (55508, 4)
data rows after drop_duplicates 26882
data row after split: 16299
data row after split: 10583
data row after split: 16299
data row after split: 10583
data row after sp

In [284]:
node = '13'
print(node)
infile = datadir + "mic/mic" + node + ".csv"

# read dataset
whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
print("data rows {}".format(whole_node_power_data.shape))
whole_node_power_data.head()


13
data rows (647583, 4)


,13,mic_id,timestamp,total_power
0,13,0,2014-03-25 19:16:36,106
1,13,1,2014-03-25 19:16:36,106
2,13,0,2014-03-25 19:16:41,106
3,13,1,2014-03-25 19:16:41,106
4,13,0,2014-03-25 19:16:46,106


In [285]:
# drop useless columns
whole_node_power_data = whole_node_power_data.drop([str(node)], axis=1)

# drop the duplicates rows (same timestamp, cpu_id, ...)
whole_node_power_data = whole_node_power_data.drop_duplicates() 
print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))
whole_node_power_data.head()



data rows after drop_duplicates 630141


,mic_id,timestamp,total_power
0,0,2014-03-25 19:16:36,106
1,1,2014-03-25 19:16:36,106
2,0,2014-03-25 19:16:41,106
3,1,2014-03-25 19:16:41,106
4,0,2014-03-25 19:16:46,106


In [286]:
interval_data = whole_node_power_data


# split the data referring to the mic0 and mic1
mic0 = interval_data[interval_data['mic_id'] == 0]
mic1 = interval_data[interval_data['mic_id'] == 1]
print("data row after split: {}".format(mic0.shape[0]))
print("data row after split: {}".format(mic1.shape[0]))
mic0.head()

data row after split: 298615
data row after split: 331526


,mic_id,timestamp,total_power
0,0,2014-03-25 19:16:36,106
2,0,2014-03-25 19:16:41,106
4,0,2014-03-25 19:16:46,106
6,0,2014-03-25 19:16:51,144
8,0,2014-03-25 19:16:56,144


In [287]:
mic1.head()

,mic_id,timestamp,total_power
1,1,2014-03-25 19:16:36,106
3,1,2014-03-25 19:16:41,106
5,1,2014-03-25 19:16:46,106
7,1,2014-03-25 19:16:51,144
9,1,2014-03-25 19:16:56,144


In [288]:
# drop the duplicates rows 
mic0 = mic0.drop_duplicates(subset=['timestamp'])
mic1 = mic1.drop_duplicates(subset=['timestamp'])
print("data row after split: {}".format(mic0.shape[0]))
print("data row after split: {}".format(mic1.shape[0]))

data row after split: 298612
data row after split: 331522


In [289]:
if mic0['timestamp'].iloc[0] < mic1['timestamp'].iloc[0]:
    mic0 = mic0[mic0.timestamp >= mic1['timestamp'].iloc[0]]
if mic1['timestamp'].iloc[0] < mic0['timestamp'].iloc[0]:
    mic1 = mic1[mic1.timestamp >= mic0['timestamp'].iloc[0]]
 

#if mic0.shape[0] < mic1.shape[0]:
#    mic1 = mic1[mic1.timestamp >= mic0['timestamp'].iloc[0]]
#if mic1.shape[0] < mic0.shape[0]:
#    mic0 = mic0[mic0.timestamp >= mic1['timestamp'].iloc[0]]
print("data row after split: {}".format(mic0.shape[0]))
print("data row after split: {}".format(mic1.shape[0]))

data row after split: 298612
data row after split: 331522


In [290]:
if mic0['timestamp'].iloc[-1] < mic1['timestamp'].iloc[-1]:
    mic1 = mic1[mic1.timestamp <= mic0['timestamp'].iloc[-1]]
if mic1['timestamp'].iloc[-1] < mic0['timestamp'].iloc[-1]:
    mic0 = mic0[mic0.timestamp <= mic1['timestamp'].iloc[-1]]
print("data row after split: {}".format(mic0.shape[0]))
print("data row after split: {}".format(mic1.shape[0]))

data row after split: 297633
data row after split: 331522


In [291]:
mic0['timestamp'].iloc[-1]

'2015-03-30 17:39:18'

In [292]:
mic1['timestamp'].iloc[-1]

'2015-03-30 17:39:28'

In [293]:
mic0['timestamp'].iloc[0]

'2014-03-25 19:16:36'

In [294]:
mic1['timestamp'].iloc[0]

'2014-03-25 19:16:36'

In [295]:
mic0.timestamp = pd.to_datetime(mic0.timestamp)
mic1.timestamp = pd.to_datetime(mic1.timestamp)
mic0 = mic0.set_index(["timestamp"])
mic1 = mic1.set_index(["timestamp"])


# resample data to 5 sec
mic0_5sec = mic0.resample('5S').mean()
mic1_5sec = mic1.resample('5S').mean()



In [296]:
mic0_5sec = mic0_5sec.drop(['mic_id'], axis=1)
mic1_5sec = mic1_5sec.drop(['mic_id'], axis=1)

In [297]:
mic0_5sec

,total_power
timestamp,
2014-03-25 19:16:35,106.0
2014-03-25 19:16:40,106.0
2014-03-25 19:16:45,106.0
2014-03-25 19:16:50,144.0
2014-03-25 19:16:55,144.0
2014-03-25 19:17:00,144.0
2014-03-25 19:17:05,144.0
2014-03-25 19:17:10,144.0
2014-03-25 19:17:15,144.0


In [298]:
mic1_5sec

,total_power
timestamp,
2014-03-25 19:16:35,106.0
2014-03-25 19:16:40,106.0
2014-03-25 19:16:45,106.0
2014-03-25 19:16:50,144.0
2014-03-25 19:16:55,144.0
2014-03-25 19:17:00,144.0
2014-03-25 19:17:05,144.0
2014-03-25 19:17:10,144.0
2014-03-25 19:17:15,144.0


In [299]:
first_date = mic0_5sec.index[0]
last_date = mic0_5sec.index[mic0_5sec.shape[0]-1]
index_before = pd.date_range(start=global_start_time, end=first_date - np.timedelta64(5, 's'), freq='5S')
index_after = pd.date_range(start=last_date + np.timedelta64(5, 's'), end=global_end_time, freq='5s')
padding_before = pd.DataFrame(index=index_before, columns=['total_power'])
padding_before.index.name = 'timestamp'
#padding_before
padding_after = pd.DataFrame(index=index_after, columns=['total_power'])
padding_after.index.name = 'timestamp'
# padding_after

mic0_5sec = pd.concat([padding_before, mic0_5sec])
mic1_5sec = pd.concat([padding_before, mic1_5sec])
mic0_5sec = pd.concat([mic0_5sec, padding_after])
mic1_5sec = pd.concat([mic1_5sec, padding_after])
print("data row after padding: {}".format(mic0_5sec.shape[0]))
print("data row after padding: {}".format(mic1_5sec.shape[0]))

data row after padding: 8743681
data row after padding: 8743683


In [300]:
mic0_5sec

,total_power
timestamp,
2014-03-24 00:00:00,NaN
2014-03-24 00:00:05,NaN
2014-03-24 00:00:10,NaN
2014-03-24 00:00:15,NaN
2014-03-24 00:00:20,NaN
2014-03-24 00:00:25,NaN
2014-03-24 00:00:30,NaN
2014-03-24 00:00:35,NaN
2014-03-24 00:00:40,NaN


In [302]:
mic1_5sec = mic1_5sec[~mic1_5sec.index.duplicated()]

In [303]:

#mic0_5sec = mic0_5sec.loc[mic0_5sec.index.drop_duplicates()]
#mic1_5sec = mic1_5sec.loc[mic1_5sec.index.drop_duplicates()]
#print("data row after padding: {}".format(mic0_5sec.shape[0]))
#print("data row after padding: {}".format(mic1_5sec.shape[0]))


mic0_5sec.rename(index=str, columns={"total_power": "pow_0"}, inplace = True)
mic1_5sec.rename(index=str, columns={"total_power": "pow_1"}, inplace = True)
print("rename")


# concat
node_5sec = pd.concat([mic0_5sec, mic1_5sec], axis=1)


node_5sec['pow_tot'] = node_5sec['pow_0'] + node_5sec['pow_1']
node_5sec = node_5sec.drop(['pow_0', 'pow_1'], axis=1)


interval_data_jobs = node_5sec[pd.to_datetime(node_5sec.index) >= start_time]
interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs.index) <= end_time]
print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))


# write
interval_comment = "Alina"
outfile = datadir + 'mic/mic' + node + "_5sec_" + interval_comment + ".csv"
interval_data_jobs.to_csv(outfile)

KeyboardInterrupt: 

In [66]:
whole_node_power_data[whole_node_power_data.timestamp == "0000-00-00 00:00:00"]

,58,gpu_id,timestamp,pow
6101257,58,1,0000-00-00 00:00:00,22.233


In [65]:
correct = pd.to_datetime(whole_node_power_data.timestamp,
                               format='%Y-%m-%d %H:%M:%S')

ValueError: time data 0000-00-00 00:00:00 doesn't match format specified